In [2]:
import importlib
import datasets
importlib.reload(datasets)
from tqdm.notebook import tqdm
from datasets import *
import random



# torch.cuda.empty_cache()

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

import clip

clip_model, clip_preprocess = clip.load("ViT-B/32", device)

cuda


In [3]:
def get_clip_features(dataset):
    all_features = []
    all_labels = []

    global clip_model

    with torch.no_grad():
        for images, labels in tqdm(dataset):
            features = clip_model.encode_image(images.to(device))
            all_features.append(features)
            all_labels.append(labels)

    return torch.cat(all_features), torch.cat(all_labels)

In [4]:
def get_text_clip_features(classes, prompt="a {} with petals."):
    # Assumes the positions are in accordance to the label numbers
    embedding_per_class = {}
    
    global clip_model

    with torch.no_grad():
        for i,_class in enumerate(classes):
            _class = _class.replace("_", " ")
            text = clip.tokenize(prompt.format(_class)).cuda() 
            class_embeddings = clip_model.encode_text(
                    text
                )
            class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
            embedding_per_class[i] = class_embeddings
    return embedding_per_class

In [ ]:
from datasets import *
dataset_obj = Flowers102(0, 50)
train_loader, _ = dataset_obj.get_train_loaders(transform_fn=clip_preprocess)
test_loader = dataset_obj.get_test_loader(transform_fn=clip_preprocess)
classes = dataset_obj.classes

In [ ]:
train_features, train_labels = get_clip_features(train_loader)
test_features, test_labels = get_clip_features(test_loader)


In [ ]:
iters = 5

In [6]:
def get_stats(l):
    return np.mean(l), np.var(l)

In [7]:
def accuracy(output, target, topk=(1,)):
    pred = output.topk(max(topk), 1, True, True)[1].t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))
    return [
        float(correct[:k].reshape(-1).float().sum(0, keepdim=True).cpu().numpy())
        for k in topk
    ]


# Baselines

### Random Embedding

In [9]:
# get zs weights 
out = []
train_loader, _ = dataset_obj.get_train_loaders(transform_fn=clip_preprocess)
train_featuress, train_labelss = get_clip_features(train_loader)
for _ in range(iters):
    for nc in [-1]:
        train_features = train_featuress / train_featuress.norm(dim=-1, keepdim=True)
        for _ in range(1):
            embeddings = []
            for c in range(len(classes)):
                ind = random.choice((c == train_labels).nonzero(as_tuple=False).tolist())
                embeddings.append(train_features[ind].cpu().numpy())

            zeroshot_weights = torch.from_numpy(np.array(embeddings).squeeze(1).T).to(torch.float16)

            def accuracy(output, target, topk=(1,)):
                pred = output.topk(max(topk), 1, True, True)[1].t()
                correct = pred.eq(target.view(1, -1).expand_as(pred))
                return [
                    float(correct[:k].reshape(-1).float().sum(0, keepdim=True).cpu().numpy())
                    for k in topk
                ]

            # lazy load
            if clip_model == None:
                clip_model, clip_preprocess = clip.load(clip_model_name, device)

            with torch.no_grad():
                top1, top5, n = 0.0, 0.0, 0.0
                for i, (images, target) in enumerate(tqdm(test_loader)):
                    images = images.cuda()
                    target = target.cuda()

                    # predict

                    image_features = clip_model.encode_image(images)
                    image_features /= image_features.norm(dim=-1, keepdim=True)
                    print(image_features.to(device).shape)
                    print(zeroshot_weights.to(device).shape)
                    logits = 100.0 * image_features.to(device) @ zeroshot_weights.to(device)

                    # measure accuracy
                    acc1, _ = accuracy(logits, target, topk=(1, 5))
                    top1 += acc1
                    n += images.size(0)

            top1 = (top1 / n) * 100

            print("num_classes:", nc, "acc:", top1)
            out.append(top1)
print(get_stats(out))

  0%|          | 0/132 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([19, 512])
torch.Size([512, 102])
num_classes: -1 acc: 1.9536019536019535


  0%|          | 0/17 [00:00<?, ?it/s]

torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([19, 512])
torch.Size([512, 102])
num_classes: -1 acc: 1.8315018315018317


...

torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([19, 512])
torch.Size([512, 102])
num_classes: -1 acc: 1.221001221001221


  0%|          | 0/17 [00:00<?, ?it/s]

torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([19, 512])
torch.Size([512, 102])
num_classes: -1 acc: 1.221001221001221


  0%|          | 0/17 [00:00<?, ?it/s]

torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([50, 512])
torch.Size([512, 102])
torch.Size([19, 512])
torch.Size([512, 102])
num_classes: -1 acc: 1.7094017094017095
(1.5873015873015874, 0.09541401482793424)


### Average Embeddings

In [9]:
# get zs weights 
for nc in [-1]:
    
    out = []
    # train_loader, _ = dataset_obj.get_train_loaders(transform_fn=clip_preprocess, num_elements_per_class=nc, shuffle=False)
    # train_featuress, train_labelss = get_clip_features(train_loader)
    for _ in range(iters):
        train_loader, _ = dataset_obj.get_train_loaders(transform_fn=clip_preprocess, num_elements_per_class=nc)
        train_features, train_labels = get_clip_features(train_loader)
        train_features /= train_features.norm(dim=-1, keepdim=True)
        for _ in range(1):
            embeddings = []
            for c in range(len(classes)):
                ind = (c == train_labels).nonzero(as_tuple=True)[0]
                embeddings.append(torch.mean(train_features[ind],dim=0))
#                 embeddings.append(np.mean(train_features[ind].cpu().numpy(), 0))

#             zeroshot_weights = torch.from_numpy(np.array(embeddings).T).to(torch.float16)
            
            zeroshot_weights = torch.stack(embeddings).squeeze(1)
            print("Done")


            # lazy load
            if clip_model == None:
                clip_model, clip_preprocess = clip.load(clip_model_name, device)

            with torch.no_grad():
                top1, top5, n = 0.0, 0.0, 0.0
                for i, (images, target) in enumerate(tqdm(test_loader)):
                    images = images.cuda()
                    target = target.cuda()

                    # predict

                    image_features = clip_model.encode_image(images)
                    image_features /= image_features.norm(dim=-1, keepdim=True)
                    logits = 100.0 * image_features.to(device) @ zeroshot_weights.to(device).T

                    # measure accuracy
                    acc1, _ = accuracy(logits, target, topk=(1, 5))
                    top1 += acc1
                    n += images.size(0)

            top1 = (top1 / n) * 100
            out.append(top1)
            print("num_classes:", nc, "acc:", top1)
    print(get_stats(out))

  0%|          | 0/1515 [00:00<?, ?it/s]

Done


  0%|          | 0/303 [00:00<?, ?it/s]

num_classes: -1 acc: 73.26072607260727


  0%|          | 0/1515 [00:00<?, ?it/s]

...

num_classes: -1 acc: 73.26072607260727


  0%|          | 0/1515 [00:00<?, ?it/s]

Done


  0%|          | 0/303 [00:00<?, ?it/s]

num_classes: -1 acc: 73.26072607260727
(73.26072607260727, 0.0)


### Nearest Embeddings

In [10]:
final = []

for _ in range(iters):
    # Normalize all the training embeddings
    train_loader, _ = dataset_obj.get_train_loaders(transform_fn=clip_preprocess)
    train_features, train_labels = get_clip_features(train_loader)
    train_features /= train_features.norm(dim=-1, keepdim=True)

    # lazy load
    if clip_model == None:
        clip_model, clip_preprocess = clip.load(clip_model_name, device)

    with torch.no_grad():
        top1, n = 0.0, 0.0
        for i, (images, targets) in enumerate(tqdm(test_loader)):
            images = images.cuda()
            targets = targets.cuda()

            # Load and normalize image features
            image_features = clip_model.encode_image(images)
            image_features /= image_features.norm(dim=-1, keepdim=True)

            for image_feature, target in zip(image_features,targets):
                # Dot product image feature with train features, find the maximum dot product(closest vector)
                out = (image_feature @ train_features.t()).argmax(dim=-1)
                closest = out.cpu().numpy()
                prediction = train_labels[closest].cpu().numpy()

                if target.cpu().numpy() == prediction:
                    top1 +=1
                n+=1

    top1 = (top1 / n) * 100
    final.append(top1)
    print("acc:", top1)

get_stats(final)

  0%|          | 0/1515 [00:00<?, ?it/s]

  0%|          | 0/303 [00:00<?, ?it/s]

acc: 74.25742574257426


  0%|          | 0/1515 [00:00<?, ?it/s]

  0%|          | 0/303 [00:00<?, ?it/s]

...

acc: 74.25742574257426


  0%|          | 0/1515 [00:00<?, ?it/s]

  0%|          | 0/303 [00:00<?, ?it/s]

acc: 74.25742574257426


(74.25742574257426, 0.0)

### Average Text and Image Embeddings

In [ ]:
text_clip_features = get_text_clip_features(classes)
for nc in [1,2,4,8,16,-1]:
    
    final = []
    for _ in range(iters):
        train_loader, _ = dataset_obj.get_train_loaders(transform_fn=clip_preprocess, num_elements_per_class=nc)
        train_features, train_labels = get_clip_features(train_loader)
        train_features /= train_features.norm(dim=-1, keepdim=True)
        for _ in range(1):
            embeddings = []
            for c in range(len(classes)):
                ind = (c == train_labels).nonzero(as_tuple=True)[0]
                image_embs = torch.mean(train_features[ind],dim=0)
                text_embs = text_clip_features[c].squeeze()
                avg_emb = 0.5*image_embs + 0.5*text_embs
                embeddings.append(avg_emb)
            
            zeroshot_weights = torch.stack(embeddings).squeeze(1)

            # lazy load
            if clip_model == None:
                clip_model, clip_preprocess = clip.load(clip_model_name, device)

            with torch.no_grad():
                top1, top5, n = 0.0, 0.0, 0.0
                for i, (images, target) in enumerate(tqdm(test_loader)):
                    images = images.cuda()
                    target = target.cuda()
                    image_features = clip_model.encode_image(images)
                    image_features /= image_features.norm(dim=-1, keepdim=True)
                    # predict
                    for image_feature,tar in zip(image_features, target):
                        big = []m
                        for c in range(len(classes)):
                            text_embs = text_clip_features[c].squeeze()
                            avg_emb = 0.5*image_feature + 0.5*text_embs
                            big.append(avg_emb)
                        big = torch.stack(big)
                        dot = big @ zeroshot_weights.T
                        out = np.unravel_index(torch.argmax(dot, axis=None).cpu().numpy(), dot.shape)[1]
                        if tar.cpu().numpy() == out:
                            top1 +=1
                        n+=1



            top1 = (top1 / n) * 100
            final.append(top1)
            print("num_classes:", nc, "acc:", top1)
    print(get_stats(final))

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/303 [00:00<?, ?it/s]

num_classes: 1 acc: 54.04620462046205


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/303 [00:00<?, ?it/s]

...

num_classes: -1 acc: 76.73927392739273


  0%|          | 0/1515 [00:00<?, ?it/s]

  0%|          | 0/303 [00:00<?, ?it/s]

num_classes: -1 acc: 76.73927392739273


  0%|          | 0/1515 [00:00<?, ?it/s]

## CLIP

In [21]:
final = []

iters = 2

for _ in range(iters):
    # Normalize all the training embeddings
    train_loader, _ = dataset_obj.get_train_loaders(transform_fn=clip_preprocess)
    train_features, train_labels = get_clip_features(train_loader)
    train_features /= train_features.norm(dim=-1, keepdim=True)
    
    train_features_per_class = {i:[] for i in range(len(classes))}
    for c in range(len(classes)):
        ind = (c == train_labels).nonzero(as_tuple=True)[0]
        class_image_embs = train_features[ind]
        train_features_per_class[c].append(class_image_embs)
                                                         
    for c in range(len(classes)):
        train_features_per_class[c] = torch.stack(train_features_per_class[c])
                                                         
    # lazy load
    if clip_model == None:
        clip_model, clip_preprocess = clip.load(clip_model_name, device)

    with torch.no_grad():
        top1, n = 0.0, 0.0
        for i, (images, targets) in enumerate(tqdm(test_loader)):
            images = images.cuda()
            targets = targets.cuda()

            # Load and normalize image features
            image_features = clip_model.encode_image(images)
            image_features /= image_features.norm(dim=-1, keepdim=True)

            for image_feature, target in zip(image_features,targets):
                # Dot product image feature with train features, find the maximum dot product(closest vector)
                prototype = []
                for c in range(len(classes)):
                    tf_c = train_features_per_class[c].squeeze(0)
                    out = (image_feature @ tf_c.t()).argmax(dim=-1)
                    closest = out.cpu().numpy()
                    prototype.append(tf_c[closest])  
                prototype = torch.stack(prototype)
                out = (image_feature @ prototype.t()).argmax(dim=-1)
                closest = out.cpu().numpy()
                prediction = closest
                
                if target.cpu().numpy() == prediction:
                    top1 +=1
                n+=1

    top1 = (top1 / n) * 100
    final.append(top1)
    print("acc:", top1)

get_stats(final)

  0%|          | 0/17 [00:00<?, ?it/s]

acc: 93.16239316239316


  0%|          | 0/17 [00:00<?, ?it/s]

acc: 93.16239316239316


(93.16239316239316, 0.0)

### Appending image and text input

In [ ]:
text_clip_features = get_text_clip_features(classes)
for nc in [-1]:
    final = []
    for _ in range(iters):
        for _ in range(1):
            embeddings = []
            for c in range(len(classes)):
                ind = (c == train_labels).nonzero(as_tuple=True)[0]
                image_embs = torch.mean(train_features[ind],dim=0)
                text_embs = text_clip_features[c].squeeze()
                app_emb = torch.cat((image_embs, text_embs), 1)
                embeddings.append(app_emb)
            
            zeroshot_weights = torch.stack(embeddings).squeeze(1)

            # lazy load
            if clip_model == None:
                clip_model, clip_preprocess = clip.load(clip_model_name, device)

            with torch.no_grad():
                top1, top5, n = 0.0, 0.0, 0.0
                for i, (images, target) in enumerate(tqdm(test_loader)):
                    images = images.cuda()
                    target = target.cuda()
                    image_features = clip_model.encode_image(images)
                    image_features /= image_features.norm(dim=-1, keepdim=True)
                    # predict
                    for image_feature,tar in zip(image_features, target):
                        big = []m
                        for c in range(len(classes)):
                            text_embs = text_clip_features[c].squeeze()
                            app_emb = torch.cat((image_feature, text_embs), 1)
                            big.append(app_emb)
                        big = torch.stack(big)
                        dot = big @ zeroshot_weights.T
                        out = np.unravel_index(torch.argmax(dot, axis=None).cpu().numpy(), dot.shape)[1]
                        if tar.cpu().numpy() == out:
                            top1 +=1
                        n+=1

            top1 = (top1 / n) * 100
            final.append(top1)
            print("num_classes:", nc, "acc:", top1)
    print(get_stats(final))

### Understanding the importance of Text embeddings 

In [10]:
from datasets import *
dataset_obj = Flowers102(0, 50)
train_loader, _ = dataset_obj.get_train_loaders(transform_fn=clip_preprocess)
test_loader = dataset_obj.get_test_loader(transform_fn=clip_preprocess)
classes = dataset_obj.classes

text_clip_features = get_text_clip_features(classes)
for nc in [-1]:
    train_loader, _ = dataset_obj.get_train_loaders(transform_fn=clip_preprocess, num_elements_per_class=nc, shuffle=False)
    train_features, train_labels = get_clip_features(train_loader)
    train_features /= train_features.norm(dim=-1, keepdim=True)
    final = []
    for lam in np.linspace(0.0, 1.0, 10):
        print(lam)

        for _ in range(1):
            text_embeds = []
            image_embeds = []
            embeddings = []
            for c in range(len(classes)):
                ind = (c == train_labels).nonzero(as_tuple=True)[0]
                image_embs = torch.mean(train_features[ind],dim=0)
                text_embs = text_clip_features[c].squeeze()
                text_embeds.append(text_embs)
                image_embeds.append(image_embs)
                avg_emb = (lam*image_embs + (1-lam)*text_embs)
                embeddings.append(avg_emb)
            
            zeroshot_weights = torch.stack(embeddings).squeeze(1)
            image_embeds = torch.stack(image_embeds).squeeze(1)
            text_embeds = torch.stack(text_embeds).squeeze(1)

            # lazy load
            if clip_model == None:
                clip_model, clip_preprocess = clip.load(clip_model_name, device)

            with torch.no_grad():
                top1, top5, n = 0.0, 0.0, 0.0
                for i, (images, target) in enumerate(tqdm(test_loader)):
                    images = images.cuda()
                    target = target.cuda()
                    image_features = clip_model.encode_image(images)
                    image_features /= image_features.norm(dim=-1, keepdim=True)
                    # predict
                    for image_feature,tar in zip(image_features, target):
                        texts_per_img = []
                        imgs_per_img = []
                        for c in range(len(classes)):
                            text_embs = text_clip_features[c].squeeze()
#                             avg_emb = 0.25(image_feature@)
                            texts_per_img.append(text_embs)
                            imgs_per_img.append(image_feature)
                        texts_per_img = torch.stack(texts_per_img)
                        imgs_per_img = torch.stack(imgs_per_img)
                        dot = lam*(imgs_per_img @ text_embeds.T) + (1-lam)*(imgs_per_img @ image_embeds.T) + 0*(texts_per_img @ text_embeds.T) + 0*(texts_per_img @ image_embeds.T)
                        out = np.unravel_index(torch.argmax(dot, axis=None).cpu().numpy(), dot.shape)[1]
                        if tar.cpu().numpy() == out:
                            top1 +=1
                        n+=1



            top1 = (top1 / n) * 100
            final.append(top1)
            print("num_classes:", nc, "acc:", top1)
    print(get_stats(final))

  0%|          | 0/132 [00:00<?, ?it/s]

0.0


  0%|          | 0/17 [00:00<?, ?it/s]

num_classes: -1 acc: 84.73748473748473
0.1111111111111111


  0%|          | 0/17 [00:00<?, ?it/s]

...

num_classes: -1 acc: 78.26617826617827
0.8888888888888888


  0%|          | 0/17 [00:00<?, ?it/s]

num_classes: -1 acc: 72.4053724053724
1.0


  0%|          | 0/17 [00:00<?, ?it/s]

num_classes: -1 acc: 66.30036630036629
(81.7094017094017, 47.71833782822797)
